Developing "Momentum" Trading Strategy
 - Imagine a train that's already chugging along. Momentum trading is a bit like hopping on when the train is speeding up.
 - People have different ways of using this strategy, but it often involves riding along with the current trend.
 - No matter how you describe it,  you first need to research potential trading signals.

In [5]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import matplotlib.pyplot as plt

1. Download data:
From Yahoo Finance API
Typically trading "systems" involve a number of securities
For this demonstration we are just going to look at GLD, the gold ETF

In [ ]:
gld = pdr.get_data_yahoo('GLD')
day = np.arange(1, len(gld) + 1)
gld['day'] = day
gld.drop(columns=['Adj Close', 'Volume'], inplace = True)
gld = gld[['day', 'Open', 'High', 'Low', 'Close']]
gld.head()

In [ ]:
gld.info()

2. Add data/transform data
We're learning how to decide when to buy or sell things like stocks. We use calculations based on prices or patterns. One method is "moving average crossover," where we calculate average prices over time. When two of these averages cross, it's a signal to act – either invest for the long term or seize short-term opportunities. Think of it like watching temperature averages to predict weather changes. We create a plan (model) to predict how this strategy could work in real situations, even though real life can be unpredictable. When the averages cross in our plan, it's a signal to make a decision based on our calculations.

In [ ]:
gld['9-day'] = gld['Close'].rolling(9).mean()
gld['21-day'] = gld['Close'].rolling(21).mean()
gld[19:25]

In [ ]:
#Add "signal" column
gld['signal'] = np.where(gld['9-day'] > gld['21-day'], 1, 0)
gld['signal'] = np.where(gld['9-day'] < gld['21-day'], -1, gld['signal'])
gld.dropna(inplace=True)
gld.head()

In [ ]:
#Calculate Instantaneous returns/system returns
gld['return'] = np.log(gld['Close']).diff()
gld['system_return'] = gld['signal'] * gld['return']
gld['entry'] = gld.signal.diff()
gld.head()

In [ ]:
#Plot trades on time series
plt.rcParams['figure.figsize'] = 12, 6
plt.grid(True, alpha = .3)
plt.plot(gld.iloc[-252:]['Close'], label = 'GLD')
plt.plot(gld.iloc[-252:]['9-day'], label = '9-day')
plt.plot(gld.iloc[-252:]['21-day'], label = '21-day')
plt.plot(gld[-252:].loc[gld.entry == 2].index, gld[-252:]['9-day'][gld.entry == 2], '^',
         color = 'g', markersize = 12)
plt.plot(gld[-252:].loc[gld.entry == -2].index, gld[-252:]['21-day'][gld.entry == -2], 'v',
         color = 'r', markersize = 12)
plt.legend(loc=2);

In [ ]:
plt.plot(np.exp(gld['return']).cumprod(), label='Buy/Hold')
plt.plot(np.exp(gld['system_return']).cumprod(), label='System')
plt.legend(loc=2)
plt.grid(True, alpha=.3)

In [ ]:
np.exp(gld['return']).cumprod()[-1] -1

In [ ]:
np.exp(gld['system_return']).cumprod()[-1] -1